<a href="https://colab.research.google.com/github/eduseiti/ia368v_dd_class_09/blob/main/DL_reranking_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TREC-COVID DL reranking fine tuning

This notebook perform reranking model fine tuning using the TREC COVID LLM queries.

## Prepare the environment

In [1]:
import os
import sys

import tqdm

In [2]:
IN_COLAB='google.colab' in sys.modules
LINK_WITH_COMET=True

In [3]:
if IN_COLAB:
    from google.colab import drive

    WORKING_FOLDER="/content/drive/MyDrive/unicamp/ia368v_dd/aula_09"

    drive.mount('/content/drive', force_remount=True)

    os.chdir(WORKING_FOLDER)
    
    !pip install transformers -q

    if LINK_WITH_COMET:
        !pip install comet_ml -q
else:
    WORKING_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/ia368v_dd_class_09/"
    PYSERINI_FOLDER="/mnt/0060f889-4c27-409b-b0de-47f5427515e3/unicamp/ia368v_dd/pyserini/"
    
    TREC_EVAL_FULLPATH=PYSERINI_FOLDER+"tools/eval/trec_eval.9.0.4/trec_eval"
    
    os.environ["ANSERINI_CLASSPATH"]="/media/eduseiti/bigdata01/unicamp/ia368v_dd/anserini/target"

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.7/201.7 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.5/506.5 kB 31.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
import pickle
import numpy as np

import json

import time

import re

from datetime import datetime

from scipy import stats

if LINK_WITH_COMET:
    from comet_ml import Experiment

In [5]:
TREC_COVID_MERGED_FILE="trec_covid_merged_data.tsv"
TREC_COVID_DOCUMENTS_FILE="trec_covid_original_title_text_merged.tsv"

TREC_COVID_QUERIES="trec_covid_queries.tsv"
TREC_COVID_QRELS="trec_covid_qrels.tsv"

API_KEYS_FILE="../api_keys_20230324.json"

pd.set_option('display.max_colwidth', None)

In [6]:
TREC_COVID_ORIGINAL_FOLDER="trec_covid_original"
TREC_COVID_ORIGINAL_INDEX_FOLDER="trec_covid_original/index"
TREC_COVID_ORIGINAL_RUNS_FOLDER="trec_covid_original/runs"

In [7]:
TREC_COVID_LLM_QUERIES="eduseiti_100_queries_expansion_20230501_01.jsonl"

In [8]:
import torch

from transformers import get_linear_schedule_with_warmup, get_constant_schedule
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils import data
from transformers import BatchEncoding

## Set the random seed

In [9]:
RANDOM_SEED = 6

rng = np.random.default_rng(RANDOM_SEED)

### Link with COMET

In [10]:
if LINK_WITH_COMET:
    with open(API_KEYS_FILE) as inputFile:
        api_keys = json.load(inputFile)

    experiment = Experiment(api_key=api_keys['comet_ml'], 
                            project_name="InPars reraking",
                            workspace="eduseiti")

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content/drive/MyDrive/unicamp/ia368v_dd/aula_09' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/eduseiti/inpars-reraking/0dfcda4ac3034586aa199a5d7b0c5b30



### Initialize reranking model parameters

In [11]:
MODEL_NAME='microsoft/MiniLM-L12-H384-uncased'
MS_MARCO_PRETRAINED_MODEL="checkpoint_0.134_20230316_234651"

MAX_TOKENS_LENGTH=512

In [12]:
TRAIN_OUTPUT_FOLDER="trained_models"

In [13]:
TREC_COVID_TOKENIZED_LLM_EXPANSION="trec_covid_tokenized_expansion_{}.pkl"

In [14]:
PYSERINI_TEST_RUN_RERANKED_FILENAME_FORMAT="run.trec_covid_reranking_{}_{}_{}.txt"

In [15]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
device

device(type='cuda')

## Prepare the fine tuning dataset

### Load the LLM generated questions

In [18]:
expanded_queries = []

with open(TREC_COVID_LLM_QUERIES, "r") as inputFile:
    for line in inputFile:
        expanded_queries.append(json.loads(line))

In [19]:
len(expanded_queries)

463

In [20]:
expanded_queries_df = pd.DataFrame(expanded_queries)

In [21]:
expanded_queries_df

,query,positive_doc_id,negative_doc_ids
0,How can chatbots be designed to effectively share up-to-date information during a pandemic?,70hskj1o,"[mt00852w, x7ol32mz, b54dymlu, h5vh6px7, bza9agzo, eumithb4, zygepxd7, i44s4vqr, 6u1zo0f9, m3a6kl8s, z9r5i0ky, c8s0jn2z, 4fb5xnil, nt7c31ft, f1ch030o, nm30wct0, qcgc2bo3, azkamnpa, enit4rki, e2g1iu39]"
1,What strategies can be used to encourage desired health impacting behaviors through chatbots?,70hskj1o,"[et84j0qi, xsfolppr, 5t2o287y, kj2tnw8q, j68x0yd1, 1lobeca0, qwdjb7vk, ue5v55l8, 3eovj63c, 411qyubx, 4qvv1hsq, j8iawzp8, 7bh268mb, gp6gz0bw, 6gc7smqf, ur9t45yc, rgeeld8q, qqsiv6r6, m7cqlzbh, hlymyzcq]"
2,What are the risks associated with amplifying misinformation with chatbot technology?,70hskj1o,"[2c1m04je, rd93y7hu, vlmvi0tf, dbq3z982, 848fswtv, uveezi7s, pat3t7ne, aimm65cr, c45feko6, gl6ozx2o, t63ni1qn, rc65rv6r, 27kfciro, pmuo5qpf, t7tjvpxv, ak97kgj5, e0nxkyhc, rh0x9gxf, idhr2upe, u75hks4k]"
3,What research has been conducted on the effectiveness of chatbots during pandemics?,70hskj1o,"[49zlztqu, amjqr9hr, hpx4723v, e790rxq9, 95bsoea2, k41xro7c, ysa8vb9x, fkv395t5, u4di2tk7, 2swzr52p, oo0z5pb2, lkzo4y8b, i6vfr6um, 8fhpsn4n, au2je08j, mi0pmyo4, fdkbuw6e, 74joo4yr, 6lrawta5, l864lrhx]"
4,"How can collaborations between healthcare workers, companies, academics and governments help prepare for future pandemics?",70hskj1o,"[eg2lj9zc, prmf9yob, ara8bsws, zjmshwl3, apvc5mml, ridgctn4, 4dv6954b, 1k168vv0, dc6jtcz0, lt67jwyv, 7ftq02ev, hbalyfy3, v9wynk5x, 22ioujwl, 6o50m9si, ti75rrwk, 3dswdn6p, jo7ty7v4, tolikanw, t1hwh3o8]"
...,...,...,...
458,What are some common symptoms of MERS-CoV infection?,24lzevco,"[jv3425w1, yz7goivp, heui8rox, optngtwu, n9pqd30o, hkvurb2k, s95ryhiu, fnrir5nh, 4hk736ev, fxck2ain, sphk023v, 345fmq8h, futlnw88, p8luczyk, fa6kbjif, zbqfs77n, 70jg65o2, jvplobgy, 0ejs05e8, m00gcci4]"
459,Are there any existing effective anti-MERS-CoV antiviral agents or therapeutics?,24lzevco,"[rigxrvhn, gxyk9fgj, 1mpov118, 08d5cdf4, eld5svt2, mg1eg740, 0wh7x410, inibtytf, rq5gh710, cmor0wkp, mpv025c6, 0lzapk68, 3mh63vjj, qg73804g, cjimyfu4, v7asfaxc, wicc796j, z0h32jyu, klx95l1j, 3fp46sov]"
460,What are some potential Host-Directed Therapies for MERS-CoV infected patients?,24lzevco,"[wtvjjc7p, mha7zs08, uwwih8v3, 1intktsf, ysbnv3fb, s8fitxwd, n3f5pihh, 4u0appfs, qexn0nuy, do9r8q84, ipmyfxk5, dkreswvk, eisfz30c, k50qvr4w, vzyrcmu4, bwnvfs8l, 6dzo97ze, n9k0ctn6, lntg6yb8, 1mowsbjy]"
461,Could Host-Directed Therapies improve treatment outcomes for patients with MERS-CoV infection?,24lzevco,"[g4oku7wp, xnjpe1ss, uqykia6i, o9uk0y2n, j5zlismf, gxi3iwb0, g2phfpbd, 5ftql1b9, k3rqx1x0, hi3fjne4, bm0ldeue, uwwih8v3, c51eyqpi, elmrvpxd, yorqoyn9, wfcyaumm, bnh65bqg, 73xuhvll, pnp8flc3, nmdko4nl]"


### Load the TREC COVID documents

In [22]:
trec_covid_docs_df = pd.read_csv(TREC_COVID_DOCUMENTS_FILE, sep='\t', header=None, names=['corpus-id', 'corpus-title-text'])

display(trec_covid_docs_df.head())

print(trec_covid_docs_df.shape)

,corpus-id,corpus-title-text
0,ug7v899j,"Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi ArabiaOBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract infections, and 2 (5%) with bronchiolitis. Cough (82.5%), fever (75%), and malaise (58.8%) were the most common symptoms, and crepitations (60%), and wheezes (40%) were the most common signs. Most patients with pneumonia had crepitations (79.2%) but only 25% had bronchial breathing. Immunocompromised patients were more likely than non-immunocompromised patients to present with pneumonia (8/9 versus 16/31, P = 0.05). Of the 24 patients with pneumonia, 14 (58.3%) had uneventful recovery, 4 (16.7%) recovered following some complications, 3 (12.5%) died because of M pneumoniae infection, and 3 (12.5%) died due to underlying comorbidities. The 3 patients who died of M pneumoniae pneumonia had other comorbidities. CONCLUSION: our results were similar to published data except for the finding that infections were more common in infants and preschool children and that the mortality rate of pneumonia in patients with comorbidities was high."
1,02tnwd4m,"Nitric oxide: a pro-inflammatory mediator in lung disease?Inflammatory diseases of the respiratory tract are commonly associated with elevated production of nitric oxide (NO•) and increased indices of NO• -dependent oxidative stress. Although NO• is known to have anti-microbial, anti-inflammatory and anti-oxidant properties, various lines of evidence support the contribution of NO• to lung injury in several disease models. On the basis of biochemical evidence, it is often presumed that such NO• -dependent oxidations are due to the formation of the oxidant peroxynitrite, although alternative mechanisms involving the phagocyte-derived heme proteins myeloperoxidase and eosinophil peroxidase might be operative during conditions of inflammation. Because of the overwhelming literature on NO• generation and activities in the respiratory tract, it would be beyond the scope of this commentary to review this area comprehensively. Instead, it focuses on recent evidence and concepts of the presumed contribution of NO• to inflammatory diseases of the lung."
2,ejv2xln0,"Surfactant protein-D and pulmonary host defenseSurfactant protein-D (SP-D) participates in the innate response to inhaled microorganisms and organic antigens, and contributes to immune and inflammatory regulation within the lung. SP-D is synthesized and secreted by alveolar and bronchiolar epithelial cells, but is also expressed by epithelial cells lining various exocrine ducts and the mucosa of the gastrointestinal and genitourinary tracts. SP-D, a collagenous calcium-dependent lectin (or collectin), binds to surface glycoconjugates expressed by a wide variety of microorganisms, and to oligosaccharides associated with the surface of various complex organic antigens. SP-D also specifically interacts with glycoconjugates and other molecules expressed on the surface of macrophages, neutrophils, and lymphocytes. In addition, SP-D binds to specific surfactant-associated lipids and can inf

(171325, 2)


### Check if hasn't already tokenized the data

In [23]:
if os.path.exists(TREC_COVID_TOKENIZED_LLM_EXPANSION.format(TREC_COVID_LLM_QUERIES)):
    with open(TREC_COVID_TOKENIZED_LLM_EXPANSION.format(TREC_COVID_LLM_QUERIES), "rb") as inputFile:
        
        tokenized_data = pickle.load(inputFile)

    trec_queries_tokens = tokenized_data['trec_queries_tokens']
    trec_docs_tokens = tokenized_data['trec_docs_tokens']
    trec_covid_selected_docs_df = tokenized_data['trec_covid_selected_docs_df']
    
    tokenized_data_read=True
else:
    tokenized_data_read=False
    
    print("Need to create the tokenized LLM queries data...")

### Build the test data to be tokenized

#### First, select only the TREC COVID documents referred in the LLM queries both as positive and negative examples

In [24]:
if not tokenized_data_read:
    selected_docs = expanded_queries_df['positive_doc_id'].unique()

    for i, row in expanded_queries_df.iterrows():
        selected_docs = np.union1d(selected_docs, row['negative_doc_ids'])

    print("selected_docs.shape: {}".format(selected_docs.shape))

    trec_covid_selected_docs_df = trec_covid_docs_df.merge(pd.DataFrame(selected_docs, columns=['doc-id']), 
                                                           left_on='corpus-id', 
                                                           right_on='doc-id', how='inner')[['corpus-id', 'corpus-title-text']]

    display(trec_covid_selected_docs_df)
else:
    print("Has already read the tokenized data...")

Has already read the tokenized data...


#### Now, tokenize both topics and returned texts

In [25]:
if not tokenized_data_read:
    trec_queries_tokens = tokenizer(expanded_queries_df['query'].tolist(), 
                                    truncation=True, 
                                    max_length=MAX_TOKENS_LENGTH, 
                                    return_length=True)

    print(stats.describe(trec_queries_tokens['length']))

    trec_docs_tokens = tokenizer(trec_covid_selected_docs_df['corpus-title-text'].tolist(), 
                                 truncation=True,
#                                  return_overflowing_tokens=True, 
                                 max_length=MAX_TOKENS_LENGTH - np.max(trec_queries_tokens['length']), 
                                 return_length=True)

    print(stats.describe(trec_docs_tokens['length']))

    #### Check if has truncated documents

    if 'overflow_to_sample_mapping' in trec_docs_tokens:    
        original_length = trec_covid_selected_docs_df.shape[0]

        if original_length < len(trec_docs_tokens['overflow_to_sample_mapping']):
            print("Added {} overflowing texts...".format(len(trec_docs_tokens['overflow_to_sample_mapping']) - original_length))


    #### Save the tokenized data

    with open(TREC_COVID_TOKENIZED_LLM_EXPANSION.format(TREC_COVID_LLM_QUERIES), "wb") as outputFile:
        pickle.dump({'trec_queries_tokens': trec_queries_tokens,
                     'trec_docs_tokens': trec_docs_tokens,
                     'trec_covid_selected_docs_df': trec_covid_selected_docs_df}, outputFile, pickle.HIGHEST_PROTOCOL)    
else:
    print("Has already read the tokenized data...")    

Has already read the tokenized data...


### Build the training dataset

In [26]:
class InParsTrainingDataset(data.Dataset):
    def __init__(self, generated_queries_df, selected_docs_df, tokenized_queries, tokenized_documents, rng):
        self.generated_queries_df = generated_queries_df
        self.selected_docs_df = selected_docs_df
        self.tokenized_queries = tokenized_queries
        self.tokenized_documents = tokenized_documents
        
        self.rng = rng
        
        self.rebuild_dataset()
        
        
    def rebuild_dataset(self, verbose=True):
        
        self.test_input_ids = []
        self.test_token_type_ids = []
        self.test_attention_mask = []
        self.labels =[]
        
        for i, row in self.generated_queries_df.iterrows():
            
            if verbose:
                print(i)
            
            selected_negative = self.rng.choice(row['negative_doc_ids'], 1)[0]

            if verbose:
                print("negative_doc_ids: {}".format(row['negative_doc_ids']))
                print("selected_negative: {}".format(selected_negative))
            
            positive_doc_index = self.selected_docs_df[self.selected_docs_df['corpus-id'] == row['positive_doc_id']].index[0]
            negative_doc_index = self.selected_docs_df[self.selected_docs_df['corpus-id'] == selected_negative].index[0]
            
            if verbose:
                print("positive_doc_index={}".format(positive_doc_index))
                print("negative_doc_index={}".format(negative_doc_index))
            
            
            self.test_input_ids.append(self.tokenized_queries['input_ids'][i] + self.tokenized_documents['input_ids'][positive_doc_index][1:])
            self.test_token_type_ids.append(self.tokenized_queries['token_type_ids'][i] + self.tokenized_documents['token_type_ids'][positive_doc_index][1:])
            self.test_attention_mask.append(self.tokenized_queries['attention_mask'][i] + self.tokenized_documents['attention_mask'][positive_doc_index][1:])
            
            self.test_input_ids.append(self.tokenized_queries['input_ids'][i] + self.tokenized_documents['input_ids'][negative_doc_index][1:])
            self.test_token_type_ids.append(self.tokenized_queries['token_type_ids'][i] + self.tokenized_documents['token_type_ids'][negative_doc_index][1:])
            self.test_attention_mask.append(self.tokenized_queries['attention_mask'][i] + self.tokenized_documents['attention_mask'][negative_doc_index][1:])
            
            self.labels.append(True)
            self.labels.append(False)
        
    
    def __len__(self):
        return len(self.test_input_ids)
    
    
    def __getitem__(self, idx):
        return {'input_ids': self.test_input_ids[idx],
                'attention_mask': self.test_attention_mask[idx],
                'labels': int(self.labels[idx])}

In [27]:
def collate_fn(batch):

    # print(len(batch[0]['input_ids']))

    r1 = tokenizer.pad(batch, return_tensors='pt')

    # print(len(r1['input_ids'][0]))

    return BatchEncoding(r1)

### Create the dataset and the dataloader

In [28]:
train_dataset = InParsTrainingDataset(expanded_queries_df,
                                      trec_covid_selected_docs_df,
                                      trec_queries_tokens,
                                      trec_docs_tokens,
                                      rng)

0
negative_doc_ids: ['mt00852w', 'x7ol32mz', 'b54dymlu', 'h5vh6px7', 'bza9agzo', 'eumithb4', 'zygepxd7', 'i44s4vqr', '6u1zo0f9', 'm3a6kl8s', 'z9r5i0ky', 'c8s0jn2z', '4fb5xnil', 'nt7c31ft', 'f1ch030o', 'nm30wct0', 'qcgc2bo3', 'azkamnpa', 'enit4rki', 'e2g1iu39']
selected_negative: 6u1zo0f9
positive_doc_index=6772
negative_doc_index=4777
1
negative_doc_ids: ['et84j0qi', 'xsfolppr', '5t2o287y', 'kj2tnw8q', 'j68x0yd1', '1lobeca0', 'qwdjb7vk', 'ue5v55l8', '3eovj63c', '411qyubx', '4qvv1hsq', 'j8iawzp8', '7bh268mb', 'gp6gz0bw', '6gc7smqf', 'ur9t45yc', 'rgeeld8q', 'qqsiv6r6', 'm7cqlzbh', 'hlymyzcq']
selected_negative: 4qvv1hsq
positive_doc_index=6772
negative_doc_index=8517
2
negative_doc_ids: ['2c1m04je', 'rd93y7hu', 'vlmvi0tf', 'dbq3z982', '848fswtv', 'uveezi7s', 'pat3t7ne', 'aimm65cr', 'c45feko6', 'gl6ozx2o', 't63ni1qn', 'rc65rv6r', '27kfciro', 'pmuo5qpf', 't7tjvpxv', 'ak97kgj5', 'e0nxkyhc', 'rh0x9gxf', 'idhr2upe', 'u75hks4k']
selected_negative: t63ni1qn
positive_doc_index=6772
negative_doc_

In [29]:
hyperparameters = {
    'batch_size': 16,
    'epochs': 3,
    'num_warmup_steps': 0,
    'learning_rate': 1e-4,
}

In [30]:
hyperparameters['num_training_steps'] = hyperparameters['epochs'] * int(len(train_dataset) // hyperparameters['batch_size'])

In [31]:
train_dataloader = data.DataLoader(train_dataset, batch_size=hyperparameters['batch_size'], shuffle=False, collate_fn=collate_fn)

In [32]:
model = AutoModelForSequenceClassification.from_pretrained(os.path.join(TRAIN_OUTPUT_FOLDER, MS_MARCO_PRETRAINED_MODEL)).to(device)
print('Parameters', model.num_parameters())

Parameters 33360770


In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=hyperparameters['learning_rate'])
scheduler = get_constant_schedule(optimizer)

In [34]:
if LINK_WITH_COMET:
    experiment.log_parameters(hyperparameters)

In [35]:
current_training_step = 0

for epoch in tqdm.tqdm(range(hyperparameters['epochs']), desc='Epochs', bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}', colour='GREEN', file=sys.stdout):
    model.train()
    train_losses = []
    
    tqdm_batches = tqdm.tqdm(train_dataloader, mininterval=0.5, desc='Train', disable=False, bar_format='{l_bar}{bar:20}{r_bar}{bar:-20b}', colour='GREEN', file=sys.stdout, position=0, leave=True)
    # tqdm_batches = tqdm.tqdm(train_dataloader, mininterval=0.5, desc='Train', disable=False)
    
    for batch in tqdm_batches:
        optimizer.zero_grad()
        outputs = model(**batch.to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_losses.append(loss.cpu().item())

        tqdm_batches.set_description("Loss {:0.4f}".format(train_losses[-1]))

        if LINK_WITH_COMET:
            experiment.log_metrics({'train loss': train_losses[-1],
                                    'learning_rate': scheduler.get_last_lr()},
                                    step=current_training_step)
        
        
    print("Epoch: {}, Training loss: {:0.4f}".format(epoch, np.mean(train_losses)))
    
    if LINK_WITH_COMET:
        experiment.log_metrics({'train loss': np.mean(train_losses)},
                               epoch=epoch)

    train_dataset.rebuild_dataset(verbose=False)

Train:   0%|                    | 0/58 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Loss 0.4045: 100%|████████████████████| 58/58 [00:35<00:00,  1.65it/s]
Epoch: 0, Training loss: 0.4254
Loss 0.3308: 100%|████████████████████| 58/58 [00:32<00:00,  1.76it/s]
Epoch: 1, Training loss: 0.2657
Loss 0.1542: 100%|████████████████████| 58/58 [00:33<00:00,  1.73it/s]
Epoch: 2, Training loss: 0.1698
Epochs: 100%|████████████████████| 3/3 [01:44<00:00, 34.96s/it]


In [36]:
if LINK_WITH_COMET:
    experiment.end() 

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/eduseiti/inpars-reraking/0dfcda4ac3034586aa199a5d7b0c5b30
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     learning_rate    : 0.0001
COMET INFO:     loss             : 0.2469821572303772
COMET INFO:     train loss [177] : (0.016603730618953705, 2.060084581375122)
COMET INFO:   Parameters:
COMET INFO:     batch_size         : 16
COMET INFO:     epochs             : 3
COMET INFO:     learning_rate      : 0.0001
COMET INFO:     num_training_steps : 171
COMET INFO:     num_warmup_steps   : 0
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packa

In [37]:
training_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

In [38]:
checkpoint_name = "checkpoint_{}_{}_{}_{:.4f}".format(TREC_COVID_LLM_QUERIES, "03_epochs", training_timestamp, np.mean(train_losses))
model.save_pretrained(os.path.join(TRAIN_OUTPUT_FOLDER, checkpoint_name))